- Grad-CAM (Gradient-weighted Class Activation Mapping)는 신경망이 특정 입력 데이터에서 어떤 부분에 주목하여 특정 클래스를 예측하는지 시각화하는 기법입
- 주로 컴퓨터 비전 모델에서 이미지 분류나 객체 검출을 할 때, 모델의 예측 근거를 이해하는 데 사용 (XAI)
- Grad-CAM은 모델의 마지막 합성곱 계층(convolutional layer)의 특성 맵(feature map)과 클래스의 기울기(gradient)를 활용해 모델의 관심 영역을 강조함으로써 표현

In [ ]:
import silence_tensorflow.auto
import os, glob
import cv2
import numpy as np
import tensorflow as tf
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

In [ ]:
# CLAHE 설정
fixed_clip = 2.0  # CLAHE의 clipLimit 설정
fixed_tile = 8    # CLAHE의 tileGridSize 설정
clahe = cv2.createCLAHE(clipLimit=fixed_clip, tileGridSize=(fixed_tile, fixed_tile))

In [ ]:
# 이미지 전처리 함수: DICOM 이미지를 불러와 CLAHE 적용, 정규화 및 리사이즈 수행
def getting_img(path_img, imageSize):
    img = sitk.ReadImage(path_img)  # DICOM 파일 읽기
    img = sitk.GetArrayFromImage(img)[0]  # 이미지 배열로 변환

    # CLAHE 적용 및 정규화
    img = clahe.apply(img)
    img = (img-img.min())/(img.max()-img.min())  # min-max normalization
    
    img_origin = img
    # 이미지 리사이즈 및 차원 확장
    img = cv2.resize(img, (imageSize, imageSize), interpolation=cv2.INTER_AREA)
    img = np.expand_dims(img, axis=-1)  # 흑백 이미지에 채널 차원 추가
    img = np.repeat(img, 3, axis=-1)    # RGB 채널로 복제하여 모델에 적합하게 변환
    return img_origin, np.expand_dims(img, axis=0)  # 배치 차원 추가

In [ ]:
# 학습된 ResNet50 기반 분류 모델 정의
def Resnet50(inputs, n_class, weights=None):
    input_x = ResNet50(include_top=False, weights=weights, input_tensor=inputs)
    
    for layer in input_x.layers:
        layer.trainable = False

    x = input_x.output
    x = GlobalAveragePooling2D(name="avg_pool")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(n_class, activation='softmax', name='output')(x)
    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
# Grad-CAM Heatmap 생성 함수
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    
    # 모델의 출력 및 마지막 컨볼루션 층의 출력을 가져오는 모델 정의
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    # 예측 및 그라디언트 계산
    with tf.GradientTape() as tape:     # tf.GradientTape: 미분을 계산하는 텐서플로우의 자동 미분 기능. 이를 이용해 관심 있는 클래스 점수에 대한 그라디언트를 기록
        last_conv_layer_output, preds = grad_model(img_array)   # 마지막 층과 출력 예측값 생성
        if pred_index is None:
            pred_index = tf.argmax(preds[0])                    # 예측값에서 가장 높은 클래스 선택
        class_channel = preds[:, pred_index]                    # 선택된 클래스의 점수만 사용

    # 그라디언트 및 풀링 계산
    grads = tape.gradient(class_channel, last_conv_layer_output)  # 선택된 클래스 점수에 대한 그라디언트 계산: class_channel (선택된 클래스 점수)와 last_conv_layer_output 사이의 그라디언트를 계산하여 grads에 저장
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))          # 풀링을 통해 전역적으로 그라디언트 요약 : 채널 차원에 대해서만 요약된 그라디언트 값 남김 
    last_conv_layer_output = last_conv_layer_output[0]            # 배치 차원 제거
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]  # 가중 그라디언트를 사용한 특성 맵 합산: pooled_grads를 각 채널에 가중치로 적용하여 last_conv_layer_output의 특성 맵을 가중합 진행 => heatmmap 생성
    heatmap = tf.squeeze(heatmap)                                 # heatmap에서 불필요한 차원을 제거하여 시각화에 적합한 형태로 변환
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap) # 정규화하여 Heatmap 생성: heatmap의 음수 값을 0으로 처리하고, 최대값으로 나누어 0에서 1 사이의 범위로 정규화
    return heatmap.numpy()

In [ ]:
# Grad-CAM 시각화 함수
def display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = cv2.cvtColor(np.uint8(255 * img), cv2.COLOR_GRAY2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    print(f'heatmap: {heatmap.shape} | img: {img.shape}')
    
    jet = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, jet, alpha, 0)
    cv2.imwrite(cam_path, superimposed_img)

    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img[..., ::-1])
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img[..., ::-1])
    plt.title("Grad-CAM")
    plt.axis("off")
    plt.show()

In [ ]:
# 모델 초기화 및 가중치 로드
n_class = 2
imageSize = 512
input_img = Input(shape=(imageSize, imageSize, 3), name='input')
model = Resnet50(input_img, n_class)

# 저장된 가중치 불러오기
path_weights = os.path.join('./Result/model_pretrain', 'last_model.h5')
model.load_weights(path_weights)

In [ ]:
# Grad-CAM 생성 및 시각화 예시
# path_data = os.path.join('/home/user/workdir/notices/data', 'breast')
path_data = os.path.join('..', '..', 'data', 'breast')
path_abimg = os.path.join(path_data, 'image', 'abnormal')
path_nrimg = os.path.join(path_data, 'image', 'normal')

path_save = os.path.join('.', 'data', 'cam_result')
os.makedirs(path_save, exist_ok=True)

abnameList = sorted(glob.glob(os.path.join(path_abimg, '*.dcm')))
nrnameList = sorted(glob.glob(os.path.join(path_nrimg, '*.dcm')))

print(len(abnameList), len(nrnameList))

labdict = {0: 'normal',
           1: 'abnormal'}

nameList, labList = [], []
nameList = abnameList+nrnameList
labList = [1 if 'abnormal' in n else 0 for n in nameList]

print(len(nameList), len(labList))
trainnames, testnames, trainlab, testlab = train_test_split(nameList, labList, test_size=0.1, random_state=4, stratify= labList)



idx_name = 0
sample_img_path = os.path.join(testnames[idx_name])
img_origin, sample_img = getting_img(sample_img_path, imageSize=imageSize)

test_result = model.predict(sample_img) # min-max 정규화

save_name = f"{os.path.basename(testnames[idx_name]).split('.')[0]}_GT{testlab[idx_name]}_Pred{test_result[0].argmax()}"
print(f"{idx_name} {save_name}")

# Grad-CAM 적용
heatmap = make_gradcam_heatmap(img_array= sample_img, 
                               model= model, 
                               last_conv_layer_name="conv5_block3_out")
display_gradcam(img = img_origin, 
                heatmap= heatmap, 
                cam_path=os.path.join(path_save , f'{save_name}.jpg'))